In [ ]:
# Instalar Selenium, BeautifulSoup, Pandas y Webdriver Manager
!pip install selenium beautifulsoup4 pandas webdriver-manager

# --- INSTALACIÓN DE GOOGLE CHROME EN COLAB ---
# 1. Descargar la clave GPG de Google Chrome
!wget -q -O - https://dl.google.com/linux/linux_signing_key.pub | sudo gpg --dearmor -o /usr/share/keyrings/google-chrome-archive-keyring.gpg

# 2. Añadir el repositorio de Google Chrome a las fuentes de apt
!echo "deb [arch=amd64 signed-by=/usr/share/keyrings/google-chrome-archive-keyring.gpg] http://dl.google.com/linux/chrome/deb/ stable main" | sudo tee /etc/apt/sources.list.d/google-chrome.list

# 3. Actualizar los listados de paquetes apt para incluir el nuevo repositorio
!sudo apt-get update

# 4. Instalar Google Chrome estable
!sudo apt-get install -y google-chrome-stable

# Opcional: Verificar la versión de Chrome instalada
!google-chrome --version

print("\n--- Instalación de Chrome y dependencias completada ---")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 63.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.8/129.8 kB 8.8 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.4.0
    Uninstalling urllib3-2.4.0:
      Successfully uninstalled urllib3-2.4.0
deb [arch=amd64 signed-by=/usr/share/keyrings/google-chrome-archive-keyring.gpg] http://dl.google.com/linux/chrome/deb/ stable main
Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://archive.ubuntu

In [ ]:
# --- PASO 1: INSTALAR LAS LIBRERÍAS NECESARIAS EN GOOGLE COLAB ---
# ¡IMPORTANTE!: Ejecuta esta celda al inicio de tu notebook.
# Si ya las tienes instaladas, Colab te dirá "Requirement already satisfied".
!pip install selenium webdriver-manager

# --- PASO 2: IMPORTAR LAS LIBRERÍAS ---
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager

def extraer_planes_claro_colab():
    url = "https://www.claro.com.pe/personas/movil/postpago/"
    planes_data = []
    processed_plans = set() # Para almacenar tuplas (nombre, precio) y evitar duplicados

    # Bandera para controlar la impresión de HTML de depuración (comentada por defecto)
    printed_debug_html = False

    # --- CONFIGURACIÓN DE SELENIUM PARA COLAB ---
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')          # Ejecuta Chrome sin abrir una ventana visual
    options.add_argument('--no-sandbox')        # Necesario para entornos Linux como Colab
    options.add_argument('--disable-dev-shm-usage') # Para evitar problemas de memoria en algunos entornos
    options.add_argument('--window-size=1920,1080') # A veces ayuda a que se carguen más elementos (resolución común)

    driver = None

    try:
        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service, options=options)
        driver.get(url)

        print("Navegador abierto en Colab y cargando la página...")

        # --- ESPERAR A QUE EL CONTENIDO DINÁMICO CARGUE ---
        # Esperar a que el contenedor principal del plan esté presente
        wait = WebDriverWait(driver, 20)
        wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'cA1PEBodyCardWrap')))

        # Dar un tiempo adicional por si acaso los elementos internos tardan un poco más en renderizarse
        time.sleep(5)

        print("Página cargada y elementos principales detectados. Extrayendo HTML...")

        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')

        plan_elements = soup.find_all('div', class_='cA1PEBodyCardWrap')

        if not plan_elements:
            print("ERROR: No se encontraron elementos con la clase 'cA1PEBodyCardWrap'.")
            print("Esto podría indicar que la clase ha cambiado nuevamente o el contenido no se cargó como se esperaba.")
            return []

        for plan_element in plan_elements:
            nombre_plan = plan_element.get('data-badge', 'N/A')

            precio_str = plan_element.get('data-price', 'N/A')
            try:
                precio = float(precio_str)
            except ValueError:
                precio = 'N/A'

            # Crear una clave única para el plan para detectar duplicados
            plan_key = (nombre_plan, precio)
            if plan_key in processed_plans:
                continue
            processed_plans.add(plan_key)

            gigas = 'N/A'
            apps_ilimitadas = 'N/A'
            llamadas_sms = 'N/A'

            # --- EXTRACCIÓN DE GIGAS ---
            gigas_tag = plan_element.find('span', class_='number')
            if gigas_tag:
                gigas_text = gigas_tag.get_text(strip=True)
                if gigas_text.upper().endswith('GB'):
                    gigas = gigas_text
                else:
                    gigas = gigas_text + ' GB'

            # --- MANEJO ESPECÍFICO PARA PLANES "MAX ILIMITADO" ---
            is_max_ilimitado_promo = False
            promo_div = plan_element.find('div', class_='cardPePromo')
            if promo_div:
                promo_text_span = promo_div.find('span', string=lambda text: text and 'Gigas, Minutos y SMS' in text)
                if promo_text_span:
                    is_max_ilimitado_promo = True
                    llamadas_sms = "Ilimitadas"
                    apps_ilimitadas = "Incluidas en Todo Ilimitado"

            # --- EXTRACCIÓN GENERAL (si no es un plan "Max Ilimitado" con la promo específica) ---
            if not is_max_ilimitado_promo:
                # Lógica para Apps Ilimitadas (para planes no 'Todo Ilimitado')
                apps_ilimitadas_list = []
                app_icon_tags = plan_element.find_all('i', class_=lambda x: x and 'cIco-rs-' in x)
                for icon_tag in app_icon_tags:
                    for cls in icon_tag.get('class', []):
                        if 'cIco-rs-' in cls:
                            app_name = cls.replace('cIco-rs-', '')
                            apps_ilimitadas_list.append(app_name.capitalize())
                apps_ilimitadas = ", ".join(apps_ilimitadas_list) if apps_ilimitadas_list else 'N/A'

                # Lógica para Llamadas y SMS (para planes no 'Todo Ilimitado')
                span_element_with_text = plan_element.find('span', string=lambda text: text and 'Llamadas y SMS' in text.strip())
                if span_element_with_text:
                    dt_parent = span_element_with_text.find_parent('dt')
                    if dt_parent:
                        dd_element = dt_parent.find_next_sibling('dd')
                        if dd_element:
                            llamadas_sms = dd_element.get_text(strip=True)

            # --- DEBUGGING: IMPRIMIR HTML DE UN PLAN "MAXILIMITADO" PARA DEPURACIÓN ---
            # Descomenta las siguientes 4 líneas si necesitas volver a depurar el HTML de un plan específico
            # if nombre_plan == 'maxilimitado69.90' and not printed_debug_html:
            #     print(f"\n--- HTML Detallado para {nombre_plan} (para depuración de Ilimitados) ---")
            #     print(plan_element.prettify())
            #     print("---------------------------------------\n")
            #     printed_debug_html = True

            planes_data.append({
                'Nombre del Plan': nombre_plan,
                'Precio (S/)': precio,
                'Gigas': gigas,
                'Apps Ilimitadas': apps_ilimitadas,
                'Llamadas y SMS': llamadas_sms
            })

    except Exception as e:
        print(f"Ocurrió un error en la ejecución de Selenium: {e}")
        print("Revisa los selectores CSS y la conexión a internet de Colab.")
        # print(driver.page_source) # Descomentar esto SOLO si necesitas ver el HTML completo de la página
    finally:
        if driver:
            driver.quit()

    # Ordenar los planes por precio antes de devolver
    planes_data_sorted = sorted(planes_data, key=lambda x: x['Precio (S/)'] if isinstance(x['Precio (S/)'], (int, float)) else float('inf'))

    return planes_data_sorted

if __name__ == "__main__":
    print("Iniciando extracción de planes en Google Colab...")
    planes = extraer_planes_claro_colab()
    if planes:
        print("\n--- Planes y Beneficios (ordenados por precio, sin duplicados) ---")
        for plan in planes:
            print(plan)

        df = pd.DataFrame(planes)
        df.to_csv("planes_claro_max_colab_ordenado_sin_duplicados.csv", index=False)
        print("\nDatos completos ordenados y sin duplicados guardados en planes_claro_max_colab_ordenado_sin_duplicados.csv")
        print("Puedes descargarlo haciendo clic en el icono de 'Archivos' (carpeta) a la izquierda en Colab.")
    else:
        print("No se pudieron extraer los planes. Revisa los mensajes de error y los selectores finales.")

Iniciando extracción de planes en Google Colab...
Navegador abierto en Colab y cargando la página...
Página cargada y elementos principales detectados. Extrayendo HTML...

--- Planes y Beneficios (ordenados por precio, sin duplicados) ---
{'Nombre del Plan': 'max29.90', 'Precio (S/)': 29.9, 'Gigas': '10 GB', 'Apps Ilimitadas': 'Facebook, Instagram, Threads, Whatsapp', 'Llamadas y SMS': 'Ilimitadas'}
{'Nombre del Plan': 'max39.90', 'Precio (S/)': 39.9, 'Gigas': '25 GB', 'Apps Ilimitadas': 'Facebook, Instagram, Threads, Whatsapp', 'Llamadas y SMS': 'Ilimitadas'}
{'Nombre del Plan': 'max49.90', 'Precio (S/)': 49.9, 'Gigas': '45 GB', 'Apps Ilimitadas': 'Facebook, Instagram, Messenger, Threads, Whatsapp', 'Llamadas y SMS': 'Ilimitadas'}
{'Nombre del Plan': 'max55.90', 'Precio (S/)': 55.9, 'Gigas': '75 GB', 'Apps Ilimitadas': 'Facebook, Instagram, Messenger, Threads, Whatsapp', 'Llamadas y SMS': 'Ilimitadas'}
{'Nombre del Plan': 'maxilimitado69.90', 'Precio (S/)': 69.9, 'Gigas': '110 GB', 'A